In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
df= pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data", header= None)


In [3]:
df.columns= ['parents', 'has_nurs','form','children','housing','finance','social','health', 'nursery']


In [4]:
df.head()


,parents,has_nurs,form,children,housing,finance,social,health,nursery
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority


In [5]:
df.describe()


,parents,has_nurs,form,children,housing,finance,social,health,nursery
count,12960,12960,12960,12960,12960,12960,12960,12960,12960
unique,3,5,4,4,3,2,3,3,5
top,usual,proper,complete,1,convenient,convenient,nonprob,recommended,not_recom
freq,4320,2592,3240,3240,4320,6480,4320,4320,4320


In [6]:
df['nursery'].value_counts()


not_recom     4320
priority      4266
spec_prior    4044
very_recom     328
recommend        2
Name: nursery, dtype: int64

In [7]:
dummies = pd.get_dummies(df.parents, drop_first= True)
 
# Concatenate the dummies to original dataframe
df = pd.concat([df, dummies], axis='columns')
 
# drop the values
df.drop(['parents'], axis='columns', inplace= True)
 
# print the dataframe
print(df)

        has_nurs      form children     housing     finance         social  \
0         proper  complete        1  convenient  convenient        nonprob   
1         proper  complete        1  convenient  convenient        nonprob   
2         proper  complete        1  convenient  convenient        nonprob   
3         proper  complete        1  convenient  convenient  slightly_prob   
4         proper  complete        1  convenient  convenient  slightly_prob   
...          ...       ...      ...         ...         ...            ...   
12955  very_crit    foster     more    critical      inconv  slightly_prob   
12956  very_crit    foster     more    critical      inconv  slightly_prob   
12957  very_crit    foster     more    critical      inconv    problematic   
12958  very_crit    foster     more    critical      inconv    problematic   
12959  very_crit    foster     more    critical      inconv    problematic   

            health     nursery  pretentious  usual  
0      rec

In [8]:
df['has_nurs'].replace(['proper', 'less_proper', 'improper', 'critical', 'very_crit'],
                        [5, 4 ,3 ,2 , 1], inplace=True)

df['form'].replace(['complete', 'completed', 'incomplete', 'foster'],
                        [ 3 ,3 ,2 , 1], inplace=True)

df['children'].replace(['more'], [4], inplace= True)
df['housing'].replace(['convenient', 'less_conv', 'critical'], [3, 2 ,1], inplace= True)
df['finance'].replace(['convenient', 'inconv'], [2 ,1], inplace= True)
df['social'].replace(['nonprob', 'slightly_prob', 'problematic'], [3, 2 ,1], inplace= True)
df['health'].replace(['recommended', 'priority', 'not_recom'], [3, 2 ,1], inplace= True)

In [9]:
nursery= df.nursery
df.drop(columns= ["nursery"] , inplace= True)

df= pd.concat([df, nursery], axis= 'columns')

In [10]:
df

,has_nurs,form,children,housing,finance,social,health,pretentious,usual,nursery
0,5,3,1,3,2,3,3,0,1,recommend
1,5,3,1,3,2,3,2,0,1,priority
2,5,3,1,3,2,3,1,0,1,not_recom
3,5,3,1,3,2,2,3,0,1,recommend
4,5,3,1,3,2,2,2,0,1,priority
...,...,...,...,...,...,...,...,...,...,...
12955,1,1,4,1,1,2,2,0,0,spec_prior
12956,1,1,4,1,1,2,1,0,0,not_recom
12957,1,1,4,1,1,1,3,0,0,spec_prior
12958,1,1,4,1,1,1,2,0,0,spec_prior


In [11]:
samples = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.3)
    samples.append((X_train, X_test, y_train, y_test))

In [12]:
samples


[(      has_nurs  form children  housing  finance  social  health  pretentious  \
  9978         4     2        1        1        2       1       3            0   
  1247         4     3        4        3        2       2       1            0   
  1187         4     3        2        1        1       1       1            0   
  2794         2     3        4        1        2       2       2            0   
  8773         5     3        3        2        2       1       2            0   
  ...        ...   ...      ...      ...      ...     ...     ...          ...   
  9347         5     1        2        3        2       2       1            0   
  3922         1     2        1        2        1       1       2            0   
  4911         5     2        3        1        1       1       3            1   
  1513         4     1        1        3        2       3       2            0   
  1551         4     1        1        1        2       2       3            0   
  
        usual

In [13]:
def fitnessfunction(X_train, X_test, y_train, y_test, kernel, c, gamma):
    model= svm.SVC( C= c, kernel= kernel, gamma=gamma)
    model.fit(X_train, y_train)
    print("fit")
    predicted = model.predict(X_test)
        
    # Model Evaluation Accuracy
    accuracy = round(np.mean(y_test == predicted) * 100, 2)
    return accuracy 
    

In [14]:
# divide dataset into training and testing sets with 10 different samples


nsamples = 10
accuracies = []
params= []
iteration= 200

kernelList = ['linear', 'rbf', 'sigmoid']
df_sample= pd.DataFrame(columns=['sample#', 'best accuracy', 'c', 'gamma', 'kernel'])

for i in range(nsamples):
    bestC = 0 
    bestGamma = 0 
    bestAccuracy=0
    X_train, X_test, y_train, y_test = samples[i]
    
    print("Iterating on sample ", i+1)
    for j in range(1, iteration+1):
        print("iteration ",j)
        kernel= np.random.choice(kernelList)
        C = np.random.uniform(0,10)
        gamma = np.random.uniform(0, 10)
        print(kernel , C, gamma)
        accuracy= fitnessfunction(X_train, X_test, y_train, y_test, kernel, C, gamma)
        if accuracy > bestAccuracy:
                bestAccuracy = accuracy
                bestC= C
                bestGamma= gamma
                bestKernel= kernel

                    
    accuracies.append(bestAccuracy)
    params.append([bestC, bestGamma, bestKernel])

    df_sample.loc[len(df_sample.index)] = [i+1, bestAccuracy, bestC, bestGamma, bestKernel]
    #print(f"Sample {i+1}: Best accuracy = {bestAccuracy}, Best params = {bestNu, bestEpsilon, bestKernel}")
print("Done iterating!\n")

Iterating on sample  1
iteration  1
sigmoid 5.185719976985741 1.4162448979211884
fit
iteration  2
rbf 2.1810056714450288 5.8246369647920195
fit
iteration  3
sigmoid 8.349120893833579 3.65943004195576
fit
iteration  4
sigmoid 2.4891124313441892 0.28934089738531066
fit
iteration  5
sigmoid 9.93232218291016 0.8558546672481027
fit
iteration  6
linear 4.53706336015267 5.1764891119577685
fit
iteration  7
sigmoid 0.7481940160117262 7.4647249791214785
fit
iteration  8
linear 6.620976263580337 0.46813146145482043
fit
iteration  9
linear 8.893927116295057 0.863643741841359
fit
iteration  10
sigmoid 0.26221681564500887 1.078106157756209
fit
iteration  11
linear 4.590452288851605 5.7442345918319475
fit
iteration  12
linear 7.711810296423918 5.884927296883707
fit
iteration  13
rbf 1.8055184725736517 4.889544159945093
fit
iteration  14
sigmoid 0.03722090366350739 6.845182711460489
fit
iteration  15
rbf 5.853897909271203 2.0907502080899976
fit
iteration  16
rbf 9.925478327863107 1.4946310669055962
fi

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

fit
iteration  181
sigmoid 0.740184760404371 8.854444439603942
fit
iteration  182
linear 8.875731575502678 8.444019667587098
fit
iteration  183
rbf 1.5299301835647616 7.038894263948677
fit


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

iteration  184
rbf 8.687667500334182 2.606396528990796
fit
iteration  185
sigmoid 6.708289559223722 8.902756594507482
fit
iteration  186
sigmoid 6.119264070639018 2.4257387255179497
fit
iteration  187
linear 2.4665023001845787 8.340168884860557
fit
iteration  188
rbf 0.08044868709517594 1.806997964317285
fit


...

fit
iteration  189
rbf 6.457687299084903 7.70055504133203
fit
iteration  190
linear 5.27953291852243 9.709989677951956
fit
iteration  191
rbf 9.974667032909132 1.9105946110834648
fit
iteration  192
sigmoid 3.394382188934002 2.598607103142898
fit
iteration  193
linear 7.7471455328475525 7.811916189147424
fit
iteration  194
sigmoid 2.335289194231284 0.4366350446679479
fit
iteration  195
rbf 4.786245131852256 1.8782745319903904
fit
iteration  196
linear 8.306942755929391 1.44425107747438


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

fit
iteration  197
rbf 9.882091908780051 1.7499023792356694
fit
iteration  198
linear 6.688240609122371 9.312993394848052
fit
iteration  199
linear 5.765618449112701 8.03805183219261
fit
iteration  200
linear 5.673379045663344 1.387356127985242
fit
Done iterating!



Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

In [15]:
print(df_sample)


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

   sample#  best accuracy         c     gamma kernel
0        1          98.95  1.306079  0.289237    rbf
1        2          98.59  7.824795  0.527138    rbf
2        3          99.02  3.940088  0.163935    rbf
3        4          98.79  4.747952  0.434881    rbf
4        5          98.77  5.947282  0.328380    rbf
5        6          98.77  7.499719  0.133451    rbf
6        7          98.92  1.524148  0.483896    rbf
7        8          99.28  1.920608  0.362394    rbf
8        9          98.82  2.329366  0.363688    rbf
9       10          98.97  3.885973  0.283815    rbf


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

In [16]:
X_train, X_test, y_train, y_test = samples[df_sample['best accuracy'].idxmax()]


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

In [17]:
from sklearn.model_selection import learning_curve
from sklearn.svm import SVC


train_sizes, train_scores, test_scores = learning_curve(SVC(kernel=df_sample['kernel'].iloc[df_sample['best accuracy'].idxmax()], 
                                                        C=df_sample['c'].iloc[df_sample['best accuracy'].idxmax()],
                                                        gamma=df_sample['gamma'].iloc[df_sample['best accuracy'].idxmax()],
                                                        max_iter = 1000), X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1,
                                                        train_sizes = np.linspace(0.01, 1.0, 50))

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

In [18]:
train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\PYTHON39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\PYTHON39\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelector